In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
plt.rcParams['figure.figsize'] = [15, 5]

In [ ]:
df = (pd.read_csv("../data/preprocessed/Building_4.csv")
      .astype({'datetime': 'datetime64'})
      .set_index('datetime'))

# Testing quality of prediction simulation

In [ ]:
temp = "outdoor_temp"
hum = "outdoor_hum"
diffuse_solar = "diffuse_solar_W_m2"
direct_solar = "direct_solar_W_m2"
pred_features = [temp, hum, diffuse_solar, direct_solar]

###################################
# change here which of the four features you want to investigate
feature = pred_features[3]
###################################

display(df.columns)

In [ ]:
#sns.pairplot(difs)
pdf = pd.concat([
    df[feature],
    df[f"pred_6h_{feature}"].shift(6),
    df[f"pred_12h_{feature}"].shift(12),
    df[f"pred_24h_{feature}"].shift(24),
], axis=1).iloc[24:, :]

In [ ]:
pred_temp_cols = [
    f'pred_6h_{feature}',
    f'pred_12h_{feature}',
    f'pred_24h_{feature}'
]

X = pdf[[f"pred_12h_{feature}", f"pred_24h_{feature}"]]
y = pdf[feature]
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
pdf_test = pdf.loc[X_test.index, :].assign(
    pred_12h_better = lr.predict(X_test),
    pred_error = lambda x: x[f"pred_6h_{feature}"] - x[feature],
    pred_better_error = lambda x: x["pred_12h_better"] - x[feature],
)

In [ ]:
t = 3354
pdf.iloc[t:t+50, :].plot()

In [ ]:
pdf_test[["pred_error", "pred_better_error"]].rename(columns={
    "pred_error": "error of 6 hour prediction from csv",
    "pred_better_error": "error of my 12 hour prediction",
    }).boxplot()

In [ ]:
print(f"   csv 24h prediction corr with actual {feature}:", pdf_test[f"pred_24h_{feature}"].corr(pdf[feature]))
print(f"   csv 12h prediction corr with actual {feature}:", pdf_test[f"pred_12h_{feature}"].corr(pdf[feature]))
print(f"   csv  6h prediction corr with actual {feature}:", pdf_test[f"pred_6h_{feature}"].corr(pdf[feature]))
print(f"fitted 12h prediction corr with actual {feature}:", pdf_test["pred_12h_better"].corr(pdf[feature]))